In [1]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import duckdb
import os
import glob
import matplotlib.pyplot as plt
import json
from tabulate import tabulate
from pathlib import Path
import seaborn as sns
from IPython.display import display

In [19]:
mlhd_path = '../data/mlhd/*/*.parquet'
musicbrainz_path = '../data/musicbrainz.db'
stat_path = 'outputs'

conn = duckdb.connect()
# conn.execute("SHOW TABLES").fetchall()
# conn.execute("DROP TABLE IF EXISTS mlhd_main")

In [20]:
conn.execute(f"SELECT COUNT(*) FROM read_parquet('{mlhd_path}')").fetchdf()
	# 20921363870

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,count_star()
0,20921363870


In [13]:
conn.close()

In [18]:
conn = duckdb.connect('../data/mlhd.db')
conn.execute("SELECT COUNT(*) FROM mlhd_plays").fetchdf()
# 20658034565

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,count_star()
0,20658034565
